**Name:** Eva Evertovna Zegelaar (2682859)

NLP Foundations

**Support for Assignment 2, 1.1:** Coreference Resolution 

## 2.1

In [1]:
import spacy
import pandas as pd
import re
nlp = spacy.load('en_core_web_sm')

In [3]:
#you should place the path to the file here
inputfile = '/Users/hernando/Desktop/NLP/port/assignment2/NLP_tech_coreference/data/bangkokpost.input.txt'

def open_read_file(filepath):
    """
    Opens and reads a text file.
    Strips URL. 
    
    :param filepath: text file or filepath
    :returns text1: string 
    """
    with open(filepath, 'r') as infile: 
        infile_read = infile.read()
        text1 = re.sub(r'http\S+', '', infile_read).strip() ##remove URL 
        return text1

infile_read = open_read_file(inputfile)

infile_read


'Coronavirus-infected Chinese tourist being treated in Thailand \n\nA Chinese tourist was found to be infected with the new strain of coronavirus when she arrived in Thailand, is being treated in hospital and is expected to be discharged in a few days, Public Health Minister Anutin Charnvirakul said on Monday.\n\nMr Anutin said the 61-year-old woman was recovering at Bamrasnaradura Infectious Diseases Institute in Nonthaburi province.\n\nShe now had no fever or any respiratory symptoms. If doctors gave her a clearance she would be allowed to go home in a few days, said Mr Anutin.\n\nSixteen other people who were close to the woman on the same flight were examined, and the results were negative, he said.\n\nMr Anutin said 59 people in China have been confirmed infected with the new strain of the coronavirus, which has been linked to a sudden outbreak of pneumonia in central China. One of them died. All had attended big markets selling animals and seafood in Wuhan city. They were either 

In [6]:
def create_simple_conll(string):
    """
    Tokenizes, enumerates tokens by sentence ID, 
    lemmatizes and pos-tags. Adds this information 
    into columns using pandas dataframe. 
    
    :param string: The string to process
    :return final_df: returns tab-separated pandas, 
    df follows CoNLL format convention. 
    """
    
    doc = nlp(string)
    tokenized = [[i.text for i in sent] for sent in doc.sents] #tokenize & sentences
    
    
    table1 = []
    for sentid, sent in enumerate(tokenized, start = 1):   #enumerate starting at 1 (conll format)
        for token in sent:
            table1.append([sentid, token])
    table2 = []
    for token in doc:                                     #lemmatize & pos tag
        lemmas = token.lemma_
        pos = token.pos_
        table2.append([lemmas, pos])
    
    
    df1 = pd.DataFrame(table1, columns=['sentid', 'token'])
    df2 = pd.DataFrame(table2, columns = ['lemmas', 'pos'])
    
    final_df = pd.concat([df1, df2], axis=1, sort=False) ## concatenate DF's
    empty_col = "_" ## conll format
    
    final_df["coreference"] = empty_col 
    return final_df



## save function as variable  
conll = create_simple_conll(infile_read)


## define the path to the outputfile here:
outputfile = 'conll.csv'
conll.to_csv(outputfile, sep='\t', index=False) 

In [7]:
conll[:20]

,sentid,token,lemmas,pos,coreference
0,1,Coronavirus,Coronavirus,PROPN,_
1,1,-,-,PUNCT,_
2,1,infected,infect,VERB,_
3,1,Chinese,chinese,ADJ,_
4,1,tourist,tourist,NOUN,_
5,1,being,be,VERB,_
6,1,treated,treat,VERB,_
7,1,in,in,ADP,_
8,1,Thailand,Thailand,PROPN,_
9,1,\n\n,\n\n,SPACE,_


In [8]:
import spacy
import neuralcoref
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

doc = nlp(u'Coronavirus-infected Chinese tourist being treated in Thailand. A Chinese tourist was found to be infected with the new strain of coronavirus when she arrived in Thailand, is being treated in hospital and is expected to be discharged in a few days, Public Health Minister Anutin Charnvirakul said on Monday. Mr Anutin said the 61-year-old woman was recovering at Bamrasnaradura Infectious Diseases Institute in Nonthaburi province.She now had no fever or any respiratory symptoms. If doctors gave her a clearance she would be allowed to go home in a few days, said Mr Anutin. Sixteen other people who were close to the woman on the same flight were examined, and the results were negative, he said. Mr Anutin said 59 people in China have been confirmed infected with the new strain of the coronavirus, which has been linked to a sudden outbreak of pneumonia in central China. One of them died. All had attended big markets selling animals and seafood in Wuhan city. They were either workers or buyers. There had not been any  human-to-human transmission of the virus. The ill Chinese woman was the first person detected with the virus outside China. Her discovery and successful treatment was indicative of the efficiency and effectiveness of health services in Thailand, Mr Anutin said. Health officials have been checking passengers from Wuhan arriving at Suvarnabhumi, Don Mueang, Phuket and Chiang Mai airports since Jan 3. They had found 12 ill passengers who justified being quarantined. Eight had so far been treated and discharged from hospital. The Chinese woman was being was treated in an isolation ward. Her infection with the new coronavirus was confirmed on Sunday, Mr Anutin said. The Public Health Ministry had not found anyone else infected with it, he said. One of Wuhan’s largest meat and seafood markets was pinpointed as the centre of the mysterious  pneumonia outbreak and was shut down on Jan. 1. The man who died had been a customer at that market. Chinese scientists identified the new virus strain last week.')
doc._.coref_clusters

[Thailand: [Thailand, Thailand],
 A Chinese tourist: [A Chinese tourist, she],
 the new strain of coronavirus: [the new strain of coronavirus, the new strain of the coronavirus, which has been linked to a sudden outbreak of pneumonia in central China],
 Public Health Minister Anutin Charnvirakul: [Public Health Minister Anutin Charnvirakul, Mr Anutin, Mr Anutin, he, Mr Anutin, Mr Anutin, Mr Anutin, he],
 the 61-year-old woman: [the 61-year-old woman, She, her, she, the woman, The ill Chinese woman, Her, The Chinese woman, Her],
 59 people in China: [59 people in China, them],
 China: [China, central China, China, the virus outside China, China],
 Wuhan: [Wuhan, Wuhan, Wuhan],
 The Public Health Ministry: [The Public Health Ministry, it]]

In [9]:
doc = nlp(u'A Chinese tourist was found to be infected with the new strain of coronavirus when she arrived in Thailand, is being treated in hospital and is expected to be discharged in a few days, Public Health Minister Anutin Charnvirakul said on Monday. Mr Anutin said the 61-year-old woman was recovering at Bamrasnaradura Infectious Diseases Institute in Nonthaburi province.She now had no fever or any respiratory symptoms. If doctors gave her a clearance she would be allowed to go home in a few days, said Mr Anutin.Sixteen other people who were close to the woman on the same flight were examined, and the results were negative, he said. Mr Anutin said 59 people in China have been confirmed infected with the new strain of the coronavirus, which has been linked to a sudden outbreak of pneumonia in central China. One of them died. All had attended big markets selling animals and seafood in Wuhan city. They were either workers or buyers. There had not been any human-to-human transmission of the virus.The ill Chinese woman was the first person detected with the virus outside China')
doc._.coref_clusters





[A Chinese tourist: [A Chinese tourist, she],
 the new strain of coronavirus: [the new strain of coronavirus, the new strain of the coronavirus, which has been linked to a sudden outbreak of pneumonia in central China],
 Public Health Minister Anutin Charnvirakul: [Public Health Minister Anutin Charnvirakul, Mr Anutin, Mr Anutin, he, Mr Anutin],
 the 61-year-old woman: [the 61-year-old woman, She, her, she, the woman, The ill Chinese woman],
 59 people in China: [59 people in China, them],
 China: [China, central China, China, China]]